In [25]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain_community.llms import CTransformers
import os
import pinecone
from pinecone import Pinecone
from dotenv import load_dotenv

In [26]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [27]:
load_dotenv()
key= os.getenv("PINECONE_API_KEY")

In [28]:
PINECONE_API_KEY = key
PINECONE_API_ENV = 'gcp-starter'

In [29]:
#extract data func
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [30]:
#path = '../'
extracted_data = load_pdf_data("../data/yt_test/bigCorps")


In [31]:
#create chunks of text
def text_chunk_splitter(extracted_data):
    text_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_split.split_documents(extracted_data)

    return text_chunk

In [32]:
text_chunks = text_chunk_splitter(extracted_data)
print("chunk length:  ", len(text_chunks))

chunk length:   2873


In [33]:
print(text_chunks[0]);print("-------")

page_content="1\n00:00:00,120 --> 00:01:01,119\nthis speaker outperformed Tony Robbins last year he was more popular and people love Tony\nRobbins it's no put in not sailing against Tony he's amazing I love the guy but people said he was\nbetter so let me ask you a question imagine you're living in your dream home you got the cabin for\nthe family the beach house your bills are paid you've built something big but you don't have your" metadata={'source': '../data/yt_test/bigCorps/gary_Big_timeSt_ytapi-v1.pdf', 'page': 0}
-------


In [34]:
#download embedding model
def download_embedding_model():
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embedding


In [35]:
embedding = download_embedding_model()

In [36]:
query_res = embedding.embed_query("Hi today is March 12 Tuesday")
print("Length: ", len(query_res))

Length:  384


In [37]:
from langchain_pinecone import PineconeVectorStore

In [38]:
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),  
   environment=os.getenv("PINECONE_ENV"),  
)
index_name = "gary-big-corps-chatbot"

## Run this line when loading the embedding to the vector store first time

In [21]:
# Run this line when loading the embedding to the vector store first time
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [39]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding)
query = "what is insulin resistance?"


from langchain.chains import RetrievalQAWithSourcesChain

retiriever = docsearch.as_retriever(search_kwargs=dict(k=3))
docs = retiriever.get_relevant_documents(query)

print("Results", docs)
#docs[0].page_content
#[doc.page_content for doc in docs]

Results [Document(page_content="effective insulin I mean insulin resistance I can't even tell you how many conditions that leads to by\nthe way do you know that we are the um one of the few civilized nations in the world that continues\nto call Alzheimer's and Dementia Alzheimer's and Dementia does anybody know what it is\n85\n00:39:49,440 --> 00:40:49,198"), Document(page_content="resistance AKA metabolic dysfunction right even the American Heart Association admits that half of\nAmericans are either pre-diabetic undiagnosed diabetes or diabetes over 150 million people\naccording to the\n77\n00:32:53,320 --> 00:33:56,599\nAmerican Heart Association have some form of insulin resistance metabolic dysfunction wow I\nwould argue it's closer to 90% but even the AHA claims it's over 150 million of 330 million plus"), Document(page_content='insulin but the industry wants to get you dependent on insulin do you know that diabetes is a 110')]


In [41]:
prompt_template = """ 
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.


Context : {context}
Question: {question}

Only return helpful answer below and nothing else.
Helpful answer:
"""

In [42]:
context = "\n\n".join([doc.page_content for doc in docs])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
#PROMPT = ChatPromptTemplate(prompt_template) #,input_variables=["context", "question"])

#prompt = PROMPT.format(context=context,question= query) 
chain_type_kwargs = {"prompt": PROMPT}

In [43]:
# from pydantic import BaseModel

# class BaseCallbackManager(BaseModel):
#     # Define your fields here
#     pass


In [44]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [45]:
from llama_cpp import Llama

## N.B
- CMAKE_ARGS="-DLLAMA_OPENBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.56

- CMAKE_ARGS="-DLLAMA_METAL_EMBED_LIBRARY=ON -DLLAMA_METAL=on" pip install -U llama-cpp-python --no-cache-dir
- updating langchain to its most uptodate solves complex bugs

In [61]:
llm = LlamaCpp(
    model_path= "../model/gguf/llama-2-7b-chat.Q3_K_M.gguf",
    n_gpu_layers= -1,
    n_batch= 512,
    max_tokens= 1024,
    n_ctx = 4096, 
    #context_length= 4096,
    temperature=0.7,
    callback_manager=callback_manager,
    #verbose=True # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model/gguf/llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

In [62]:
# qa_sources = RetrievalQAWithSourcesChain.from_chain_type(
#     llm=llm, 
#     retriever = retiriever,
#     chain_type="stuff",
#     chain_type_kwargs=chain_type_kwargs,
#     return_source_documents=True
# )
QA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retiriever,
    #retriever = docsearch.as_retriever(search_kwargs={'k':3}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs
)


In [63]:
import textwrap

In [64]:
query = "what is insulin resistance?"
result = QA({'query': query})
result['result']

Insulin resistance is a condition where the body's cells become less responsive to insulin, a hormone produced by the pancreas that regulates blood sugar levels. Insulin resistance occurs when the body's cells are unable to use insulin effectively, leading to high blood sugar levels. It can be caused by various factors, including genetics, obesity, physical inactivity, and certain medical conditions. Insulin resistance can lead to conditions such as prediabetes and type 2 diabetes, which can increase the risk of heart disease, stroke, and other health problems if left untreated.


llama_print_timings:        load time =   20089.28 ms
llama_print_timings:      sample time =      63.20 ms /   137 runs   (    0.46 ms per token,  2167.58 tokens per second)
llama_print_timings: prompt eval time =   20087.24 ms /   383 tokens (   52.45 ms per token,    19.07 tokens per second)
llama_print_timings:        eval time =   33731.84 ms /   136 runs   (  248.03 ms per token,     4.03 tokens per second)
llama_print_timings:       total time =   54943.57 ms /   519 tokens


"Insulin resistance is a condition where the body's cells become less responsive to insulin, a hormone produced by the pancreas that regulates blood sugar levels. Insulin resistance occurs when the body's cells are unable to use insulin effectively, leading to high blood sugar levels. It can be caused by various factors, including genetics, obesity, physical inactivity, and certain medical conditions. Insulin resistance can lead to conditions such as prediabetes and type 2 diabetes, which can increase the risk of heart disease, stroke, and other health problems if left untreated."

In [65]:
import sys
while True:
    user_input = input(f"Input prompt: ")
    if user_input == 'exit':
        print("Exiting")
        sys.exit()
    if user_input == ' ':
        continue
    result = QA({'query': user_input})
    result = textwrap.fill(result["result"], width=50)
    #print(f"Answer: {wraped_result['wraped_result']}")
    print(f"Answer: {result}")

Input prompt:  what solves insulin resistance?


Llama.generate: prefix-match hit


Insulin resistance is a condition where the body's cells become less responsive to insulin, a hormone produced by the pancreas that helps regulate blood sugar levels. When cells become resistant to insulin, the body produces more insulin in an attempt to keep blood sugar levels under control, leading to a vicious cycle of insulin resistance and insulin overproduction.
There are several factors that can contribute to insulin resistance, including:
1. Genetics: Insulin resistance can be inherited from one's parents.
2. Obesity: Excess body fat, particularly around the abdominal area, can contribute to insulin resistance.
3. Physical inactivity: A sedentary lifestyle can lead to insulin resistance.
4. Poor diet: Consuming high amounts of refined carbohydrates, sugar, and unhealthy fats can contribute to insulin resistance.
5. Hormonal imbalances: Certain hormonal imbalances, such as polycystic ovary syndrome (PCOS), can contribute to insulin resistance.
6. Medications: Certain medications


llama_print_timings:        load time =   20089.28 ms
llama_print_timings:      sample time =     451.54 ms /  1024 runs   (    0.44 ms per token,  2267.80 tokens per second)
llama_print_timings: prompt eval time =    7270.24 ms /   178 tokens (   40.84 ms per token,    24.48 tokens per second)
llama_print_timings:        eval time =  270087.99 ms /  1023 runs   (  264.02 ms per token,     3.79 tokens per second)
llama_print_timings:       total time =  290771.34 ms /  1201 tokens


Answer: Insulin resistance is a condition where the body's
cells become less responsive to insulin, a hormone
produced by the pancreas that helps regulate blood
sugar levels. When cells become resistant to
insulin, the body produces more insulin in an
attempt to keep blood sugar levels under control,
leading to a vicious cycle of insulin resistance
and insulin overproduction. There are several
factors that can contribute to insulin resistance,
including: 1. Genetics: Insulin resistance can be
inherited from one's parents. 2. Obesity: Excess
body fat, particularly around the abdominal area,
can contribute to insulin resistance. 3. Physical
inactivity: A sedentary lifestyle can lead to
insulin resistance. 4. Poor diet: Consuming high
amounts of refined carbohydrates, sugar, and
unhealthy fats can contribute to insulin
resistance. 5. Hormonal imbalances: Certain
hormonal imbalances, such as polycystic ovary
syndrome (PCOS), can contribute to insulin
resistance. 6. Medications: Certain med

Input prompt:  exit


Exiting


SystemExit: 

/Users/mtm007/Downloads/LLM/Medchatbot/RAG_MedChatBot/Rchat/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# while True:
#     user_Input: input(f"Input prompt: ")
#     result = QA({"query": user_Input})
#     print("Response: ", result["result"])

- ValueError: Document prompt requires documents to have metadata variables: ['source']. Received document with missing metadata: ['source'].

In [56]:

qa_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    retriever = retiriever,
    chain_type="stuff",
    #chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [59]:
#qa_sources(query)----refer to the above value error, the source document needs improved data ingestion and procesing toninclude metadata like source